In [ ]:
from arango import ArangoClient
client = ArangoClient(hosts="http://localhost:8529")
sys_db = client.db("_system")

In [ ]:
db = client.db("test-profiling")

In [ ]:
class Function:
    db = None
    call_edges = None
    lookups = {}
    
    @classmethod
    def clear_colls(cls):
        for fn in cls.lookups.values():
            fn.coll.delete_match({})
        cls.call_edges.delete_match({})

    def __init__(self, name, *calls):
        self.coll = (
            self.db.collection(name) 
            if self.db.has_collection(name) 
            else self.db.create_collection(name)
    )
        self._id = None
        self.caller = None
        self.lookups[name] = self
        for fn in calls:
            fn.caller = self
    
    def store_log_entry(self, **doc):
        self._id = self.coll.insert(doc)["_id"]
        if self.caller is not None:
            self.call_edges.insert({
                "_from" : self.caller._id,
                "_to" : self._id
            })

In [ ]:
Function.db = db
Function.call_edges = db.collection("callsFunc")

In [ ]:
Function("event", 
    Function("fitAndStore",
        Function("vertexFit",
             Function("fitGFRaveVertex")
        ),
        Function("removeHitsAndRefit")
    )
)

In [ ]:
Function.clear_colls()

In [ ]:
def log_extract(filepath):
    from utils import readlines_reversed
    fns = Function.lookups
    normal = ["event", "vertexFit", "fitGFRaveVertex", "removeHitsAndRefit"]
    for line in filter(
        lambda s: "NewV0Fitter::" in s or "V0FinderModule::" in s,
        readlines_reversed(filepath)
    ):
        tokens = line.split()[2:]
        for k in normal:
            if k in line:
                print(k)
                [t, *_] = tokens
                fns[k].store_log_entry(t=float(t))
                break
        if (k:="fitAndStore") in line:
            [_, ptype, t, *_] = tokens
            fns[k].store_log_entry(ptype=ptype[:-1], t=float(t))

In [ ]:
log_extract("v0finder-validation.log")

In [ ]:
event = {
    "t" : 0,
    "fitAndStore" : [
        {
            "t" : 0,
            "ptype" : "",
            
            "vertexFit" : [
                {
                    "t" : 0,
                    "fitGFRaveVertex":"",
                }
            ],
            
            "removeHitsAndRefit" : {
                "t" : 0
            }
        }
    ]
}

In [ ]:
db.collections()

In [ ]:
coll = db.collection("event")

In [ ]:
res = coll.insert({"foo": "bar"})

In [ ]:
res

In [ ]:
coll.delete("event/4984")

In [ ]:
coll.delete_match({})

In [ ]:
coll.all().count()

In [ ]:
"""
FOR e_call in event
  LET t_fas = SUM(
    FOR fas_call IN 1..1 OUTBOUND e_call callsFunc
      RETURN fas_call.t
  )
  RETURN { t_e : e_call.t , t_fas : t_fas }
"""